In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/netflix-shows/netflix_titles.csv


# Data Science Project: Data Wrangling
    Name : Shirleen Simon
    Date : 23rd May 2025
This project involves data cleaning and transformation of Netflix dataset from Kaggle.
The various steps are : Discovery,Structuring,Cleaning,Enriching,Validating and lastly Publishing.

In [2]:
import pandas as pd
import numpy as np
import datetime as dt

# Import the data
df = pd.read_csv('/kaggle/input/netflix-shows/netflix_titles.csv')


# Step 1 : _Discovery_

The initial phase where you explore and understand the data to determine its structure, content, and quality

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [4]:
# Number of rows and columns
print("Shape of the dataset (R x C):", df.shape)

Shape of the dataset (R x C): (8807, 12)


In [5]:
# List of all column names
print("Columns in the dataset:\n", df.columns.tolist())

Columns in the dataset:
 ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description']


In [6]:
# Data types of each column
print("Data types:\n", df.dtypes)

Data types:
 show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object


In [7]:
# Group and Count of missing (null) values in each column
print("Missing values per column:\n", df.isnull().sum())

Missing values per column:
 show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64


In [8]:
# Group and Count of duplicate rows
print("Number of duplicate rows:", df.duplicated().sum())

Number of duplicate rows: 0


# Step 2: _Structuring_

It involves transforming raw data into a consistent and usable format for analysis.

In [9]:
# Convert 'date_added' to datetime
df['date_added'] = pd.to_datetime(df['date_added'],format='mixed')

In [10]:
# Separate 'duration' into numeric value and unit (e.g., '90 min' → 90, 'min')
df[['duration_value', 'duration_unit']] = df['duration'].str.extract(r'(\d+)\s*(\w+)')

In [11]:
# Convert duration_value to numeric
df['duration_value'] = pd.to_numeric(df['duration_value'])

In [12]:
# View Resulting columns
print(df[['duration_value', 'duration_unit']])

      duration_value duration_unit
0               90.0           min
1                2.0       Seasons
2                1.0        Season
3                1.0        Season
4                2.0       Seasons
...              ...           ...
8802           158.0           min
8803             2.0       Seasons
8804            88.0           min
8805            88.0           min
8806           111.0           min

[8807 rows x 2 columns]


# Step 3: _Cleaning_

Involves improving data quality by correcting errors, inconsistencies, and missing values

In [13]:
# Check for duplicate rows
print("Duplicate rows before:", df.duplicated().sum())

Duplicate rows before: 0


In [14]:
# Drop duplicate rows if any
df = df.drop_duplicates()

In [15]:
# Drop description column because it will not be used
df = df.drop(columns=['description'])

In [16]:
# Impute Director values by using relationship between cast and director

In [17]:
# List of Director-Cast pairs and the number of times they appear
df['dir_cast'] = df['director'] + '---' + df['cast']
counts = df['dir_cast'].value_counts() #counts unique values
filtered_counts = counts[counts >= 3] #checks if repeated 3 or more times
filtered_values = filtered_counts.index #gets the values i.e. names
lst_dir_cast = list(filtered_values) #convert to list
dict_direcast = dict()
for i in lst_dir_cast:
    director, cast = i.split('---')
    dict_direcast[director] = cast

for i in range(len(dict_direcast)): 
    df.loc[(df['director'].isna()) & (df['cast'] == list(dict_direcast.items())[i][1]),'director'] = list(dict_direcast.items())[i][0]

In [18]:
# Assign Not Given to all other director fields
df.loc[df['director'].isna(),'director'] = 'Not Given'


In [19]:
#Use directors to fill missing countries
directors = df['director']
countries = df['country']
#pair each director with their country use zip() to get an iterator of tuples
pairs = zip(directors, countries)
# Convert the list of tuples into a dictionary
dir_cntry = dict(list(pairs))

In [20]:
# Director matched to Country values used to fill in null country values
for director, country in dir_cntry.items():
    df.loc[(df['country'].isna()) & (df['director'] == director), 'country'] = country
# Assign Not Given to all other country fields
df.loc[df['country'].isna(), 'country'] = 'Not Given'

In [21]:
df.loc[df['cast'].isna(), 'cast'] = 'Not Given'


In [22]:
# dropping other row records that are null
df.drop(df[df['date_added'].isna()].index,axis=0,inplace=True)
df.drop(df[df['rating'].isna()].index,axis=0,inplace=True)
df.drop(df[df['duration'].isna()].index,axis=0,inplace=True)

# Step 4: _Error_ / _Enriching_

During this process, errors and outliers that come with raw data are identified, corrected, and removed.After data is explored and processed, it needs to be enriched-the process of enhancing, refining, and improving raw data.

<details>
<summary>Click to expand manual revert steps</summary>

```python
# Step 1: Identify indices
reverted_indices = [1551, 1696, 2920, 3168, 3954, 4054, 4381, 4412, 4467, 4491, 4541, 4543, 4568, 4577]

# Step 2: Revert changes
df.loc[reverted_indices, 'release_year'] = df.loc[reverted_indices, 'date_added'].dt.year + 1

# Step 3: Confirm inconsistencies are back
print("Inconsistent count after revert:", sum(df['date_added'].dt.year < df['release_year']))


In [23]:
# check if there are any added_dates that come before release_year
import datetime as dt
sum(df['date_added'].dt.year < df['release_year'])

14

In [24]:
# View any added_dates that come before release_year
sum(df['date_added'].dt.year < df['release_year'])
df.loc[df['date_added'].dt.year < df['release_year'], ['title', 'date_added', 'release_year']]


,title,date_added,release_year
1551,Hilda,2020-12-14,2021
1696,Polly Pocket,2020-11-15,2021
2920,Love Is Blind,2020-02-13,2021
3168,Fuller House,2019-12-06,2020
3287,Maradona in Mexico,2019-11-13,2020
3369,BoJack Horseman,2019-10-25,2020
3433,The Hook Up Plan,2019-10-11,2020
4844,Unbreakable Kimmy Schmidt,2018-05-30,2019
4845,Arrested Development,2018-05-29,2019
5394,Hans Teeuwen: Real Rancour,2017-07-01,2018


In [25]:
# Fix: set release_year to date_added year where date_added < release_year
mask = df['date_added'].dt.year < df['release_year']
df.loc[mask, 'release_year'] = df.loc[mask, 'date_added'].dt.year

In [26]:
# sample some of the records and check that they have been accurately replaced
df.iloc[[1551,1696,2920,3168]]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,duration_value,duration_unit,dir_cast
1551,s1552,TV Show,Hilda,Not Given,"Bella Ramsey, Ameerah Falzon-Ojo, Oliver Nelso...","United Kingdom, Canada, United States",2020-12-14,2020,TV-Y7,2 Seasons,Kids' TV,2.0,Seasons,NaN
1696,s1697,TV Show,Polly Pocket,Not Given,"Emily Tennant, Shannon Chan-Kent, Kazumi Evans...","Canada, United States, Ireland",2020-11-15,2020,TV-Y,2 Seasons,Kids' TV,2.0,Seasons,NaN
2920,s2921,TV Show,Love Is Blind,Not Given,"Nick Lachey, Vanessa Lachey",United States,2020-02-13,2020,TV-MA,1 Season,"Reality TV, Romantic TV Shows",1.0,Season,NaN
3168,s3169,TV Show,Fuller House,Not Given,"Candace Cameron Bure, Jodie Sweetin, Andrea Ba...",United States,2019-12-06,2019,TV-PG,5 Seasons,TV Comedies,5.0,Seasons,NaN


In [27]:
#Confirm that no more release_year inconsistencies
sum(df['date_added'].dt.year < df['release_year'])

0

# Step 5 : _Validate_

 The process of verifying that data adheres to predefined rules, standards, and expectations.

In [28]:
#Remove any columns you may have added during wrangling
df['dir_cast'] = df['director'] + '---' + df['cast']
#Ensure each column has the correct data type e.g. verify that date_added is datetime and duration_value is numeric.
print(df.dtypes)
print(df.isnull().sum())
#Use business logic or sanity rules to identify anomalies e.g. records before 1997
print(df[df['release_year'] < 1997])  # Sanity check
#Sample a few rows to check visually e.g. df.sample(5)
print(df.sample(5))


show_id                   object
type                      object
title                     object
director                  object
cast                      object
country                   object
date_added        datetime64[ns]
release_year               int64
rating                    object
duration                  object
listed_in                 object
duration_value           float64
duration_unit             object
dir_cast                  object
dtype: object
show_id           0
type              0
title             0
director          0
cast              0
country           0
date_added        0
release_year      0
rating            0
duration          0
listed_in         0
duration_value    0
duration_unit     0
dir_cast          0
dtype: int64
     show_id   type                                title          director  \
7         s8  Movie                              Sankofa      Haile Gerima   
22       s23  Movie                      Avvai Shanmughi    K.S. Ravikumar 

In [29]:
#Reset the Index
df = df.reset_index(drop=True)


# Step 6: _Publishing_

The final stage where the cleaned, transformed, and validated data is made available for analysis, reporting, or other use.

In [30]:
# Save as CSV 
df.to_csv('/kaggle/working/cleaned_netflix.csv', index=False)